In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import torch
from torch import nn

In [2]:
mechanism=1 #1:random, 2:mostclose, 3:worst for worst change nn_pick to 1
d = {1:'random',2:'mostclose',3:'worst'}
experiment_name = f'nn-normal-{d[mechanism]}'
path = Path('../../data/results/experiment1')

In [3]:
df = pd.read_csv(path/'best_known.csv')
best_known_fitness = df['fitness'].values
best_known_sumcv = df['sum_constraints'].values

In [4]:
D = 30
runs = 30
nn_window = 5
nn_nf = 4
frequency = 1_000
max_times =100 
total_generations = max_times * frequency + 1000

In [5]:
class SimpleModel(nn.Module):
    def __init__(self, d=D, w=nn_window, nf=nn_nf):
        super().__init__()
        self.fc1 = nn.Linear(d,nf)
        self.fc2 = nn.Linear(nf*w,d)
        self.act = nn.ReLU(inplace=True)
        
    def forward(self, x):
        fts = torch.cat([self.fc1(x[:,i]) for i in range(x.size(1))], dim=1)
        return self.fc2(self.act(fts))

In [6]:
nn_pick = 3 # how many individuals we predict and replace(only for worst mechanism i put to 1)
nn_window = 5 # time window used to predict next optimal


if mechanism==1:
    replace_mechanism = ReplaceMechanism.Random
elif mechanism==2:
    replace_mechanism = ReplaceMechanism.Closest
elif   mechanism==3:
    replace_mechanism = ReplaceMechanism.Worst

In [7]:
experiment=1


if experiment==1:
    ab = pd.read_csv(path/'dC_01.csv')['b'].values
    def fitness_func(indiv, b, t): return (indiv.data**2).sum()
    def constraint_func(indiv, b, t): return -b[t] + sum((1/np.sqrt(D))*indiv.data)
    
elif experiment==2:
    ab = pd.read_csv(path/'dC_01.csv')['b'].values
    def fitness_func(indiv, b, t): return (indiv.data**2).sum()
    def constraint_func(indiv, b, t): return -b[t] + sum((1/np.sqrt(D))*indiv.data)
elif experiment==3:
    ab=[]
    # experiment 3: dynamic fitness function (linear)
    def fitness_func(indiv, b, t): return ((indiv.data + 0.1*t)**2).sum()
    def constraint_func(indiv, b, t): return 0    
else:
    ab=pd.read_csv(path/'dC_01.csv')['b'].values
# experiment 5: dynamic fitness function (x-sin(pi/2*t))^2 with random changes on peak of sin
    def fitness_func(indiv, b, t): return ((indiv.data-b[t]*np.sin(np.pi/2*t))**2).sum()
    def constraint_func(indiv, b, t): return 0

In [ ]:
results = {'mof':[], 'sr':[], 'nfe':[], 'fitness':[], 'sumcv':[], 'arr':[], 'nn_time':[]}

for run in range(runs):
    model = SimpleModel()
    population = Population.new_random(dimension=D)
    nn_trainer = partial(NNTrainer, model=model, n=nn_pick, window=nn_window, replace_mechanism=replace_mechanism)

    speed_metric = partial(SpeedMetric, threadhold=0.2)

    opt = Optimization(population, fitness_func, constraint_func, fitness_params=ab, constraint_params=[ab],
                       max_times=max_times, frequency=frequency, callbacks=[nn_trainer],
                       metrics=[speed_metric, ModifiedOfflineError, OfflineError, AbsoluteRecoverRate],
                       optimal_fitness_values=best_known_fitness, optimal_sum_constraints=best_known_sumcv)
    opt.state_dict
    opt.run(total_generations)
    # saving results
    results['mof'].append(opt.modified_offline_error.metrics)
    results['arr'].append(opt.absolute_recover_rate.metrics)
    results['sr'].append(opt.speed_metric.metrics)
    results['nfe'].append(opt.speed_metric.speeds)
    results['fitness'].append(opt.recorder.best_times_fitness)
    results['sumcv'].append(opt.recorder.best_times_constraints) 
    results['nn_time'].append(opt.nn_timer.metrics)

Total time: 01:55
SpeedMetric(success rate): 0.3700
ModifiedOfflineError: 1.7431
OfflineError: 1.7337
AbsoluteRecoverRate: 1.5644
NNTimer: 0.1549


In [ ]:
df = pd.DataFrame({'fitness':opt.recorder.best_times_fitness, 'sum_constraints':opt.recorder.best_times_constraints})
df.head()

In [ ]:
opt.recorder.plot();

In [ ]:
opt.recorder.plot_metrics();

In [ ]:
pd.DataFrame({'mof':results['mof']}).to_csv(path/f'{experiment_name}_mof.csv', index=False)
pd.DataFrame({'sr':results['sr']}).to_csv(path/f'{experiment_name}_sr.csv', index=False)
pd.DataFrame(results['nfe']).to_csv(path/f'{experiment_name}_nfe.csv', index=False)
pd.DataFrame(results['fitness']).to_csv(path/f'{experiment_name}_fitness.csv', index=False)
pd.DataFrame(results['sumcv']).to_csv(path/f'{experiment_name}_sumcv.csv', index=False)
pd.DataFrame(results['arr']).to_csv(path/f'{experiment_name}_arr.csv', index=False)
pd.DataFrame(results['nn_time']).to_csv(path/f'{experiment_name}_nn_time.csv', index=False)